In [2]:
import matplotlib.pyplot as plt
import numpy.ma as ma
import xarray as xr
import rioxarray as rxr
from shapely.geometry import mapping, box
import geopandas as gpd
import rasterio 
from rasterio import plot
import numpy as np


In [6]:
import os 
tifs = [doc for doc in os.listdir("D:\\bulk_download_USGS\\Bulk_Order_Turkana_2\\Landsat_8-9_OLI_TIRS_C2_L2") if doc.endswith(".tiff")]
files = [item for item in os.listdir("D:\\bulk_download_USGS\\Bulk_Order_Turkana_2\\Landsat_8-9_OLI_TIRS_C2_L2") if item not in tifs]

In [15]:
for i in range(35,36): 
    print (i)

35


To Delete extra files and calculate NDVI for each image

In [19]:
count = 1
directory = "D:\\bulk_download_USGS\\Bulk_Order_Turkana_2\\Landsat_8-9_OLI_TIRS_C2_L2\\"
# for i in range(len(files)): 
for i in range(35, 36):
    # delete extraneous files 
    direct = directory + files[i]
    # all files in the directory
    l8_files = os.listdir(direct)
    # files we want to keep 
    keep_files = [file for file in l8_files if file.endswith("B4.TIF") or file.endswith("B5.TIF") or file.endswith("B2.TIF")]
    # files we want to delete 
    to_delete = [item for item in l8_files if item not in keep_files]
    # delete 
    for file in to_delete:
        path_to_file = os.path.join(direct, file)
        os.remove(path_to_file)
    
    #define file paths 
    b4file_path = directory + files[i] + "\\" + files[i] + "_SR_B4.TIF"
    b5file_path = directory + files[i] + "\\" + files[i] + "_SR_B5.TIF"
    
    # open files
    b4 = rasterio.open(b4file_path)
    b5 = rasterio.open(b5file_path)
    
    print (b4, b5)
    # change dtype 
    red = b4.read().astype('float64')
    nir = b5.read().astype('float64')

    
    # calculate NDVI 
    ndvi = np.where(
        (nir+red)==0.,
        -255,
        (nir-red)/(nir+red)
        )
    
    # write raster file
    new_path = directory + files[i] + "_NDVI.tiff"
    ndviImage = rasterio.open(new_path, 'w', 
                         driver = 'Gtiff', width = b4.width, height = b4.height, count=1, crs = b4.crs, 
                         transform = b4.transform, dtype = 'float64')

    ndviImage.write(ndvi,1)
    ndviImage.close()
    
    print (str(count), "/", str(len(files)))
    count += 1

<open DatasetReader name='D:/bulk_download_USGS/Bulk_Order_Turkana_2/Landsat_8-9_OLI_TIRS_C2_L2/LC08_L2SP_170058_20160513_20200907_02_T1/LC08_L2SP_170058_20160513_20200907_02_T1_SR_B4.TIF' mode='r'> <open DatasetReader name='D:/bulk_download_USGS/Bulk_Order_Turkana_2/Landsat_8-9_OLI_TIRS_C2_L2/LC08_L2SP_170058_20160513_20200907_02_T1/LC08_L2SP_170058_20160513_20200907_02_T1_SR_B5.TIF' mode='r'>


RasterioIOError: Read or write failed. D:/bulk_download_USGS/Bulk_Order_Turkana_2/Landsat_8-9_OLI_TIRS_C2_L2/LC08_L2SP_170058_20160513_20200907_02_T1/LC08_L2SP_170058_20160513_20200907_02_T1_SR_B4.TIF, band 1: IReadBlock failed at X offset 25, Y offset 12: TIFFReadEncodedTile() failed.

## CALCULATE IMPORTANT STATISTICS FOR NDVI CORRELATION 


In [3]:
# pull all NDVI files
ndvi_path = "C:\\Tara_Fall_2019\\Kenya_Drought\\Calculated_NDVI"
import os 
ndvi_files = os.listdir(ndvi_path)

In [10]:
ndvi_files[1][17:-28]

'20140201'

Crop NDVI files since many of them have slightly different extents which will throw off calculations

In [4]:
crop_extent = gpd.read_file("C:\\Tara_Fall_2019\\Kenya_Drought\\CLIP.shp")
for file in ndvi_files: 
    inpath = "C:\\Tara_Fall_2019\\Kenya_Drought\\Calculated_NDVI"
    outpath = "C:\\Tara_Fall_2019\\Kenya_Drought\\Calculated_NDVI\\" + file[:-5] + "CLIP.tif"
    raster = rxr.open_rasterio(os.path.join(inpath, file), masked=True).squeeze()

    raster_clipped = raster.rio.clip(crop_extent.geometry.apply(mapping),
                                      # This is needed if your GDF is in a diff CRS than the raster data
                                      crop_extent.crs)
    raster_clipped.rio.to_raster(outpath)

group files by footprint and year 

In [16]:
# open all files for a given year 
f58_14 = [file for file in ndvi_files if "170058_2014" in file and "CLIP" in file]
f58_15 = [file for file in ndvi_files if "170058_2015" in file and "CLIP" in file]
f58_16 = [file for file in ndvi_files if "170058_2016" in file and "CLIP" in file]
f58_17 = [file for file in ndvi_files if "170058_2017" in file and "CLIP" in file]
f58_18 = [file for file in ndvi_files if "170058_2018" in file and "CLIP" in file]
f58_19 = [file for file in ndvi_files if "170058_2019" in file and "CLIP" in file]
f58_20 = [file for file in ndvi_files if "170058_2020" in file and "CLIP" in file]
f58_21 = [file for file in ndvi_files if "170058_2021" in file and "CLIP" in file]


ndvi_lists = [f58_14, f58_15, f58_16, f58_17, f58_18, f58_19, f58_20, f58_21]

In [76]:
f58_19[0]
ndvi_path = "C:\\Tara_Fall_2019\\Kenya_Drought\\Calculated_NDVI"
ar = rasterio.open(os.path.join(ndvi_path, f58_19[0])).read(1).astype('float64')

In [80]:
ar[]

nan

In [74]:
# calculate maximum NDVI for each pixel in a given year 

# for each year in the record
for l in ndvi_lists: 
    # specify path 
    ndvi_path = "C:\\Tara_Fall_2019\\Kenya_Drought\\Calculated_NDVI"
    # select the ndvi list 
    ndvi_l = l
    
    # save info from one tif to write new file 
    b4 = rasterio.open(os.path.join(ndvi_path, ndvi_l[0]))
    
    # create an empty list to save the arrays of data
    arrays = []
    
    # for each tif in the ndvi list for the year
    for tif in ndvi_l: 
        #append to the list of ararys 
        arrays.append(rasterio.open(os.path.join(ndvi_path, tif)).read(1).astype('float64'))
        
    # calculate max and min ndvi with reduce function for all ndvi tifs over the year
    max_ndvi = np.maximum.reduce(arrays)
    min_ndvi = np.minimum.reduce(arrays)
    
    # write raster files
    new_path_max = "C:\\Tara_Fall_2019\\Kenya_Drought\\Annual_NDVI_min_max\\" + ndvi_l[0][14:-32] + "_NDVI_max.tiff"
    new_path_min = "C:\\Tara_Fall_2019\\Kenya_Drought\\Annual_NDVI_min_max\\" + ndvi_l[0][14:-32] + "_NDVI_min.tiff"

    ndvimax = rasterio.open(new_path_max, 'w', 
                         driver = 'Gtiff', width = b4.width, height = b4.height, count=1, crs = b4.crs, 
                         transform = b4.transform, dtype = 'float64')
    ndvimax.write(max_ndvi,1)
    ndvimax.close()
    
    ndvimin = rasterio.open(new_path_min, 'w', 
                         driver = 'Gtiff', width = b4.width, height = b4.height, count=1, crs = b4.crs, 
                         transform = b4.transform, dtype = 'float64')
    ndvimin.write(min_ndvi,1)
    ndvimin.close()
    print ("done", ndvi_l[0][0][14:-32])

done 
done 
done 
done 
done 
done 
done 
done 


## Workspace below 

In [17]:
arrays = []
for tif in f58_15: 
    arrays.append(rasterio.open(os.path.join(ndvi_path, tif)).read(1))

In [38]:
max_ndvi = np.maximum.reduce([arrays[0], arrays[1], arrays[2], arrays[3], arrays[4], arrays[5], arrays[9]])

In [51]:
max_ndvi[2500][2500]

0.3958038090629429

In [52]:
arrays[0].flatten().shape

(31746318,)